In [1]:
!pip install frontend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [10]:
!pip uninstall fitz --y

In [11]:
!pip install PyMuPDF

In [12]:
#!pip install gradio
!pip install gradio

In [13]:
!pip install crewai

In [15]:
import os
import fitz  # PyMuPDF
from crewai import Crew, Agent, Task
from textwrap import dedent
from langchain_openai import ChatOpenAI
import openai
import gradio as gr



# Directly set the OpenAI API key
openai_api_key = 'api_key'

# Initialize OpenAI API key
openai.api_key = openai_api_key

# Define Resume Optimization Agents
class ResumeOptimizationAgents:
    def __init__(self):
        self.OpenAIGPT4 = ChatOpenAI(model_name="gpt-4", openai_api_key=openai_api_key, temperature=0.7)

    def expert_resume_optimizer(self):
        return Agent(
            role="Expert Resume Optimizer",
            backstory=dedent(
                """Expert in optimizing resumes to improve chances of passing automated screenings and getting noticed by recruiters.
                I have decades of experience in resume building and career consulting."""),
            goal=dedent("""
                        Optimize resumes to improve chances of passing automated screenings and getting noticed by recruiters.
                        """),
            verbose=True,
            llm=self.OpenAIGPT4
        )

    def resume_analysis_expert(self):
        return Agent(
            role="Resume Analysis Expert",
            backstory=dedent("""Expert at analyzing resumes for strengths and weaknesses.
                Skilled in identifying areas for improvement."""),
            goal=dedent("""Analyze the given resume for strengths and weaknesses and provide feedback."""),
            verbose=True,
            llm=self.OpenAIGPT4
        )

    def job_description_analysis_expert(self):
        return Agent(
            role="Job Description Analysis Expert",
            backstory=dedent("""Expert at analyzing job descriptions to identify key requirements and keywords."""),
            goal=dedent("""Analyze the given job description to identify key requirements and keywords."""),
            verbose=True,
            llm=self.OpenAIGPT4
        )

    def keyword_optimization_expert(self):
        return Agent(
            role="Keyword Optimization Expert",
            backstory=dedent("""Expert at optimizing resumes with relevant keywords to match job descriptions."""),
            goal=dedent("""Optimize the resume by adding relevant keywords from the job description."""),
            verbose=True,
            llm=self.OpenAIGPT4
        )

# Define Resume Optimization Tasks
class ResumeOptimizationTasks:
    def analyze_resume(self, agent, resume_text):
        return Task(
            description=dedent(
                f"""
            **Task**: Analyze Resume
            **Description**: Evaluate the given resume for strengths and weaknesses. Identify areas for improvement and provide feedback on how to enhance the resume's effectiveness. Focus on structure, clarity, and relevance to the target job.

            **Parameters**:
            - Resume Text: {resume_text}

            **Note**: Provide detailed and actionable feedback to help improve the resume.
            """
            ),
            agent=agent,
            expected_output="A detailed analysis of the resume, highlighting strengths and areas for improvement."
        )

    def analyze_job_description(self, agent, job_description):
        return Task(
            description=dedent(
                f"""
            **Task**: Analyze Job Description
            **Description**: Review the given job description to identify key requirements and keywords. Highlight the most important skills, experiences, and attributes that the employer is seeking.

            **Parameters**:
            - Job Description: {job_description}

            **Note**: Provide a list of key requirements and keywords extracted from the job description.
            """
            ),
            agent=agent,
            expected_output="A list of key requirements and keywords extracted from the job description."
        )

    def optimize_resume(self, agent, resume_text, job_description):
        return Task(
            description=dedent(
                f"""
            **Task**: Optimize Resume with Keywords
            **Description**: Optimize the resume by incorporating relevant keywords and phrases from the job description. Ensure that the resume aligns with the job requirements and improves its chances of passing automated screening processes.

            **Parameters**:
            - Resume Text: {resume_text}
            - Job Description: {job_description}

            **Note**: Provide a final optimized resume that is tailored to the job description.
            """
            ),
            agent=agent,
            expected_output="An optimized resume tailored to the job description, incorporating relevant keywords and phrases."
        )

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Main function to run the Resume Optimization Crew
class ResumeCrew:
    def __init__(self, resume_text, job_description):
        self.resume_text = resume_text
        self.job_description = job_description

    def run(self):
        agents = ResumeOptimizationAgents()
        tasks = ResumeOptimizationTasks()

        expert_resume_optimizer = agents.expert_resume_optimizer()
        resume_analysis_expert = agents.resume_analysis_expert()
        job_description_analysis_expert = agents.job_description_analysis_expert()
        keyword_optimization_expert = agents.keyword_optimization_expert()

        analyze_resume_task = tasks.analyze_resume(resume_analysis_expert, self.resume_text)
        analyze_job_description_task = tasks.analyze_job_description(job_description_analysis_expert, self.job_description)
        optimize_resume_task = tasks.optimize_resume(keyword_optimization_expert, self.resume_text, self.job_description)

        crew = Crew(
            agents=[expert_resume_optimizer, resume_analysis_expert, job_description_analysis_expert, keyword_optimization_expert],
            tasks=[analyze_resume_task, analyze_job_description_task, optimize_resume_task],
            verbose=True,
        )

        result = crew.kickoff()
        return result

# Gradio interface
def optimize_resume_interface(pdf_file, job_description):
    resume_text = extract_text_from_pdf(pdf_file.name)
    print("Resume Text:", resume_text[:500])  # Print the first 500 characters for debugging
    print("Job Description:", job_description)  # Print the job description for debugging
    resume_crew = ResumeCrew(resume_text, job_description)
    result = resume_crew.run()
    print("Optimized Resume:", result)  # Print the optimized resume for debugging
    return result

# Gradio interface setup
iface = gr.Interface(
    fn=optimize_resume_interface,
    inputs=[
        gr.File(label="Upload your resume (PDF)"),
        gr.Textbox(label="Paste the job description")
    ],
    outputs="text",
    title="Resume Optimization",
    description="Upload your resume and provide a job description to get an optimized resume."
)

if __name__ == "__main__":
    iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ad936e4284ad3a8746.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
